In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [2]:
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [4]:
from langchain.chat_models import init_chat_model
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain_mongodb import MongoDBAtlasVectorSearch
from pymongo import MongoClient
from pymongo.server_api import ServerApi

MONGO_USER = os.getenv("MONGO_USER")
MONGO_PASSWORD = os.getenv("MONGO_PASSWORD")
MONGO_CLUSTER = os.getenv("MONGO_CLUSTER")
MONGO_DATABASE = os.getenv("MONGO_DATABASE")

uri = f"mongodb+srv://{MONGO_USER}:{MONGO_PASSWORD}@{MONGO_CLUSTER}.igv4jfg.mongodb.net/?retryWrites=true&w=majority&appName={MONGO_CLUSTER}"

client = MongoClient(uri, server_api=ServerApi('1'))
db = client.get_database(MONGO_DATABASE)

rev_embeddings_collection = db["review_embeddings"]

loader = TextLoader("docs/fake_internships_info.md")
docs = loader.load()


splitter = MarkdownHeaderTextSplitter(headers_to_split_on=[
    ("####", "company"), ("###", "overview"), ("##", "comments"), ("#", "sources")
])

split_docs = splitter.split_text(docs[0].page_content)


llm = init_chat_model(model="llama3-8b-8192", model_provider="groq")
embeddings = HuggingFaceEmbeddings(model="sentence-transformers/all-MiniLM-L6-v2")

def return_embeddings(string: str) -> list[float]:
    embed = embeddings.embed_query(string)
    return embed

data = []
current_company = None
current_section = None

for content in split_docs:
    print(content) 


page_content='Company: Codsoft' metadata={'company': ''}
page_content='Overview: CodSoft is also a fake company on Linkedin which hires people as interns. The newbies feel that they are excited but their cover letter is also automated, their whole hiring process is automated and everything. Their tasks are just automated provided on a pdf or sheet to perform and it's useless to do internship if you are a newbie. Here are some comments from users in Reddit:' metadata={'overview': ''}
page_content='Comments: It's fake and for certificate u need to pay i guess It's useless, Codsoft is so fake' metadata={'comments': ''}
page_content='Sources from Reddit: https://www.reddit.com/r/internships/comments/1k8esbw/is_codsoft_intership_are_fake_or_real/' metadata={'sources': ''}
page_content='Company: GaoTek or Gao RFID Inc.' metadata={'sources': '', 'company': ''}
page_content='Overview: GaoTek Inc or Gao RFID Inc is a virtual company based in US or Canada. They are known for their fake internshi